In [2]:
from dotenv import load_dotenv

import pandas as pd
import matplotlib.pyplot as plt

# Load environment variables
load_dotenv()

print("Setup complete. Environment variables loaded and libraries imported.")

Setup complete. Environment variables loaded and libraries imported.


In [3]:
# Load the master dataset with all statistics
df_original = pd.read_csv('../data/new/master_translated.csv')
print(f"Loaded master dataset with {len(df_original):,} rows.")

Loaded master dataset with 302,442 rows.


In [4]:
# Drop duplicate answer_ids to ensure each answer is only counted once
print(f"Before dropping duplicates, dataset has {len(df_original):,} rows.")
df_original = df_original.drop_duplicates(subset=['answer_id'])
print(f"After dropping duplicates, dataset has {len(df_original):,} rows.")



Before dropping duplicates, dataset has 302,442 rows.
After dropping duplicates, dataset has 269,424 rows.


In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from transformers.trainer_utils import get_last_checkpoint
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


In [6]:
# Example: get top 50 users by answer count
user_answer_counts = df_original['user_id'].value_counts()
top_50_users = user_answer_counts.head(50)

# Reverse the order of the top 50 users
top_50_users = top_50_users.iloc[::-1]

print("Top 50 users by number of answers:")
for user_id, count in top_50_users.items():
    print(f"User {user_id}: {count:,} answers")


Top 50 users by number of answers:
User 22606: 501 answers
User 32329: 509 answers
User 22853: 510 answers
User 51978: 522 answers
User 16297: 525 answers
User 52981: 533 answers
User 52584: 535 answers
User 22601: 536 answers
User 50149: 538 answers
User 53005: 540 answers
User 52977: 544 answers
User 51618: 546 answers
User 52913: 546 answers
User 16289: 552 answers
User 52995: 554 answers
User 51184: 570 answers
User 52979: 577 answers
User 52171: 579 answers
User 53024: 596 answers
User 52989: 597 answers
User 52912: 602 answers
User 52991: 616 answers
User 30844: 635 answers
User 52174: 646 answers
User 52295: 647 answers
User 52423: 650 answers
User 52937: 653 answers
User 52311: 680 answers
User 23076: 685 answers
User 52167: 695 answers
User 52914: 698 answers
User 52645: 703 answers
User 52209: 723 answers
User 52173: 743 answers
User 52407: 751 answers
User 51986: 761 answers
User 53007: 807 answers
User 52170: 817 answers
User 52933: 897 answers
User 50061: 899 answers
User 

In [7]:
def train_user_model(
    df: pd.DataFrame,
    user_id: int,
    model_name: str = "answerdotai/ModernBERT-base",
    output_dir_base: str = "user_models",
    push_to_hub: bool = False,
    hub_repo_org: str = None,
    hub_token: str = None,
    num_train_epochs: int = 5,
    batch_size: int = 4
):
    """
    Trains a ModernBERT model to predict correctness for a single user.

    Args:
        df (pd.DataFrame): Original DataFrame with user data.
        user_id (int): Which user's data to train on.
        model_name (str): Hugging Face model ID for ModernBERT.
        output_dir_base (str): Base directory where user-specific checkpoints are stored.
        push_to_hub (bool): Whether to push final model to Hugging Face Hub.
        hub_repo_org (str): Org or username for HF Hub repositories.
        hub_token (str): Personal access token if required for private repos.
        num_train_epochs (int): Number of training epochs.
        batch_size (int): Training batch size.
    """

    # --------------------------
    # 1) Filter user data
    # --------------------------
    df_user = df[df["user_id"] == user_id].copy()
    if df_user.empty:
        print(f"[User {user_id}] No data. Skipping.")
        return

    # Convert is_correct to an integer label
    df_user["label"] = df_user["is_correct"].astype(int)

    # --------------------------
    # 2) Combine text: question + choices + topics
    # --------------------------
    def combine_text(row):
        q = str(row.get("question_title", ""))
        a = str(row.get("option_a", ""))
        b = str(row.get("option_b", ""))
        c = str(row.get("option_c", ""))
        d = str(row.get("option_d", ""))
        e = str(row.get("option_e", ""))
        topic = str(row.get("topic_name", ""))
        subj = str(row.get("subject_name", ""))
        axis = str(row.get("axis_name", ""))

        return (
            f"Topic: {topic}\n"
            f"Subject: {subj}\n"
            f"Axis: {axis}\n\n"
            f"Question: {q}\n"
            f"A) {a}\n"
            f"B) {b}\n"
            f"C) {c}\n"
            f"D) {d}\n"
            f"E) {e}"
        )
    
    df_user["text"] = df_user.apply(combine_text, axis=1)

    # --------------------------
    # 3) Train/Test Split
    # --------------------------
    train_df, test_df = train_test_split(
        df_user, 
        test_size=0.2, 
        shuffle=True, 
        random_state=42
    )

    if train_df.empty or test_df.empty:
        print(f"[User {user_id}] Not enough data to split. Skipping.")
        return

    # Build HF Datasets
    train_dataset_hf = Dataset.from_pandas(train_df[["text","label"]].reset_index(drop=True))
    test_dataset_hf = Dataset.from_pandas(test_df[["text","label"]].reset_index(drop=True))

    ds_dict = DatasetDict({
        "train": train_dataset_hf,
        "test": test_dataset_hf
    })

    # --------------------------
    # 4) Tokenize
    # --------------------------
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    def tokenize_fn(batch):
        return tokenizer(
            batch["text"],
            padding="max_length",
            truncation=True,
            max_length=512
        )
    
    ds_dict = ds_dict.map(tokenize_fn, batched=True)

    # --------------------------
    # 5) Load Model
    # --------------------------
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2
    )
    model.config.problem_type = "single_label_classification"

    # --------------------------
    # 6) Training Arguments
    # --------------------------
    user_output_dir = os.path.join(output_dir_base, f"user_{user_id}")
    os.makedirs(user_output_dir, exist_ok=True)

    # Optional: define the Hub repo ID if pushing
    if push_to_hub:
        if hub_repo_org:
            hub_repo_id = f"{hub_repo_org}/modernbert-user-{user_id}"
        else:
            # If no org specified, the model will be pushed to your personal account
            hub_repo_id = f"modernbert-user-{user_id}"
    else:
        hub_repo_id = None

    training_args = TrainingArguments(
        output_dir=user_output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_train_epochs,
        use_mps_device=True,              # Apple Silicon
        bf16=True,                        # optional, can cause issues on some M2 setups
        optim="adamw_torch_fused",        # fused optimizer for speed
        logging_steps=50,
        load_best_model_at_end=True,
        metric_for_best_model="f1",       # or "accuracy"
        save_total_limit=2,              # keep only last 2 checkpoints
        push_to_hub=push_to_hub,         # to decide if we push
        hub_model_id=hub_repo_id,        # if we push
        hub_token=hub_token,             # if needed
        hub_strategy="end",              # push only at end or each save
    )

    # Define metrics
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, preds)
        f1 = f1_score(labels, preds, average="weighted")
        return {"accuracy": acc, "f1": f1}

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds_dict["train"],
        eval_dataset=ds_dict["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # --------------------------
    # 7) Resume Check
    # --------------------------
    last_checkpoint = None
    if os.path.isdir(user_output_dir):
        last_checkpoint = get_last_checkpoint(user_output_dir)
    if last_checkpoint is not None:
        print(f"[User {user_id}] Resuming from {last_checkpoint}")
    else:
        print(f"[User {user_id}] Starting from scratch...")

    # --------------------------
    # 8) Train
    # --------------------------
    trainer.train(resume_from_checkpoint=last_checkpoint)

    # The best checkpoint is already loaded at end (load_best_model_at_end=True)
    # Evaluate final
    eval_metrics = trainer.evaluate()
    print(f"[User {user_id}] Final eval metrics:", eval_metrics)

    # --------------------------
    # 9) Save final model
    # --------------------------
    # This ensures we save *just* the best
    trainer.save_model(user_output_dir)

    # If pushing to hub, trainer will push after the final save (hub_strategy="end").
    # Alternatively, you can manually call:
    # if push_to_hub:
    #     trainer.push_to_hub()

    print(f"[User {user_id}] Done! Best model saved at: {user_output_dir}")


In [ ]:
# Directory where you store user-specific models locally
OUTPUT_DIR_BASE = "user_models"

for user_id, count in top_50_users.items():
    print(f"\n=== Training model for User {user_id} (Answers={count}) ===")
    train_user_model(
        df=df_original,
        user_id=user_id,
        model_name="answerdotai/ModernBERT-base",
        output_dir_base=OUTPUT_DIR_BASE,
        push_to_hub=True,               # Set to False if you don't want to push
        hub_token=os.getenv("HUGGINGFACE_ACCESS_TOKEN"),
        num_train_epochs=10,            # Adjust as needed
        batch_size=4                   # Adjust for your M2 memory constraints
    )



=== Training model for User 22606 (Answers=501) ===


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/101 [00:00<?, ? examples/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.12/site-packages/transformers/training_args.py:2241: UserWarning: `use_mps_device` is deprecated and will be removed in version 5.0 of 🤗 Transformers. `mps` device will be used by default if available similar to the way `cuda` device is used.Therefore, no action from user is required. 
  warnings.warn(
/var/folders/58/5cdkrpys1rv_tbq6wq1x69vc0000gq/T/ipykernel_13394/2454030636.py:158: FutureWarning: `tokenizer` 

[User 22606] Resuming from user_models/user_22606/checkpoint-1000


...

  0%|          | 0/1290 [00:00<?, ?it/s]

{'loss': 0.8784, 'grad_norm': 3.485852003097534, 'learning_rate': 4.8062015503875976e-05, 'epoch': 0.39}
{'loss': 0.7015, 'grad_norm': 8.442926406860352, 'learning_rate': 4.6124031007751936e-05, 'epoch': 0.78}


  0%|          | 0/33 [00:00<?, ?it/s]

{'eval_loss': 0.7320469617843628, 'eval_accuracy': 0.6384615384615384, 'eval_f1': 0.4975803539183821, 'eval_runtime': 5.4588, 'eval_samples_per_second': 23.815, 'eval_steps_per_second': 6.045, 'epoch': 1.0}
{'loss': 0.669, 'grad_norm': 4.5944976806640625, 'learning_rate': 4.418604651162791e-05, 'epoch': 1.16}
{'loss': 0.6395, 'grad_norm': 3.814293384552002, 'learning_rate': 4.2248062015503877e-05, 'epoch': 1.55}
{'loss': 0.6333, 'grad_norm': 4.437618255615234, 'learning_rate': 4.0310077519379843e-05, 'epoch': 1.94}


  0%|          | 0/33 [00:00<?, ?it/s]